In [ ]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_ollama import OllamaLLM

example_template = """
Input:
{input}

Output:
{output}
"""

examples = [
    {
        "input": "El que quiera saber de qué color son las penas siente plaza de soldado y auséntese de su tierra",
        "output": "El que quiera saber\nde qué color son las penas\nsiente plaza de soldado\ny auséntese de su tierra"
    },
    {
        "input": "Me dices que era un gato el que entró por tu ventana. En la vida he visto yo, serrano de mi querer, un gato negro y con sotana.",
        "output": "Me dices que era un gato\nel que entró por tu ventana.\nEn la vida he visto yo,\nserrano de mi querer,\nun gato negro y con sotana."
    }
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""You are a lyrics segmenter. Your task is to take unformatted song lyrics and divide them into meaningful verses.

    Each verse should be separated by **one newline**.
    Keep the original wording — do not invent lyrics.
    Here are some examples:""",
    suffix="""Now segment the following song into verses. Only return the song with each verse on a new line (no extra information like syllables or any other data):
    
    Input:
    {input}

    Output (verses separated by one newline):""",
    input_variables=["input"],
)

llm = OllamaLLM(model="segmenter2_llama2", params={"format": "json"})

#input_song = "Que rueden, que rueden, las cáscaras de huevo: las lavanderas hacen así, las planchadoras hacen así, las barrenderas hacen así"
input_song = "Y al aura el gentil capullo busca al imán el acero la fuente el que tiene sed y mi corazón al tuyo Y con esto me despido adiós público del alma al que le guste mi canto mil gracias da la gitana,"
final_prompt = few_shot_prompt.format(input=input_song)

result = llm.invoke(final_prompt)

print(result)

ResponseError: "jina/jina-embeddings-v2-base-es" does not support generate (status code: 400)

In [18]:
from langchain.prompts import ChatPromptTemplate

reflection_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a lyrics segmenter. You must check how well the segmentation has been applied in this song."
        "This is the original song: {lyrics}. This is the current segmentation: {segmented_lyrics}"
        "Is the segmentation logical? Do the verses flow well? Are there poetic or stylistic patterns? Are improvements possible? Is the format correct? Are the verses separated by **one newline**?"
        "Do the lyrics match? The lyrics must be the same in the original song and the segmented song. Change the current segmentation if necessary."
        "Do the syllables count match each verse's syllable count? Count again and change the current Syllables count if necessary"
        "Each verse should be separated by **one newline**. Keep the original wording — do not invent lyrics."
    )
])

reflection_input = reflection_prompt.format_messages(segmented_lyrics=result, lyrics=input_song)
reflected_result = llm.invoke(reflection_input)

print(reflected_result)

{
"Verses": [
"Y al aura el gentil capullo",
"busca al imán",
"el acero",
"el que tiene sed",
"mi corazón al tuyo",
"Y con esto me despido",
"adiós público del alma",
"al que le guste mi canto",
"mil gracias da la gitana"
],
"Syllables": [10, 9, 5, 7, 8, 8, 7, 6]
}


In [ ]:
from langchain.prompts import ChatPromptTemplate

reflection_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a lyrics segmenter. You must check how well the segmentation has been applied in this song. The following is a response given by an AI to a prompt:"
        "This is the original song: {lyrics}. This is the current segmentation given by an AI: {segmented_lyrics}"
        "Is the segmentation logical? Do the verses flow well? Are there poetic or stylistic patterns? Are improvements possible? Is the format correct? Are the verses separated by **one newline**?"
        "Change the current segmentation if necessary."
        "Do the syllables count match each verse's syllable count? Count again and change the current Syllables count if necessary"
        "Each verse should be separated by **one newline**. Keep the original wording — do not invent lyrics."
    )
])

reflection_input = reflection_prompt.format_messages(segmented_lyrics=reflection_input, lyrics=input_song)
reflected_result = llm.invoke(reflection_input)

print(reflected_result)


{
"Verses": [
"Y al aura el gentil capullo",
"busca al imán",
"el acero",
"el que tiene sed",
"mi corazón al tuyo",
"Y con esto me despido",
"adiós público del alma",
"al que le guste mi canto",
"mil gracias da la gitana"
],
"Syllables": [
4, 5, 7, 8, 8, 8, 9, 10
]
}

The segmentation is logical and the verses flow well. There are poetic and stylistic patterns in the song, such as the use of repetition and rhyme. The format is correct, and each verse is separated by one newline. The syllable count matches each verse's syllable count, and the lyrics match the original song.

However, there are some improvements that could be made to the segmentation:

* The first line of the second verse ("busca al imán") could be separated by a newline before the word "el", making the flow of the verses smoother.
* The last line of the seventh verse ("al que le guste mi canto") could be shortened to fit the rhyme scheme better, such as "al que me gusta cantar".
* The eighth verse ("Y con esto me despi